In [1]:
# ==========================================
# File: project_rdkt/notebooks/01_feature_engineering.ipynb
# ==========================================
# [RDKit 专属] 特征工程 (健壮性增强版)
# 核心改进:
# 1. 结构聚合: 对重复 SMILES 的 Target 取均值，而不是直接丢弃
# 2. 健壮切分: 增加分箱失败的兜底逻辑，防止报错
# 3. 索引保护: Imputer 处理后强制还原 DataFrame 格式，确保索引对齐
# 4. 脏数据防御: 增加对非字符串 SMILES 的过滤

import os
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

# 1. 环境准备
if 'notebooks' in os.getcwd(): os.chdir('..')
print(f"🚀 启动 RDKit 特征工程 (Robust Mode) | 目录: {os.getcwd()}")

# 元数据记录
metadata = {
    "step": "01_feature_engineering",
    "metrics": {}
}

# ========================================================
# [Step 1] 读取数据 & 识别列名
# ========================================================
print("\n>>> [Step 1] Loading Raw Data...")
raw_dir = 'data/raw'
if not os.path.exists(raw_dir): raise FileNotFoundError(f"❌ 找不到 {raw_dir}")

# 按文件名排序读取，保证顺序一致
csv_files = sorted([f for f in os.listdir(raw_dir) if f.lower().endswith('.csv')])
if not csv_files: raise FileNotFoundError(f"❌ {raw_dir} 为空")

file_path = os.path.join(raw_dir, csv_files[0])
df_raw = pd.read_csv(file_path)
print(f"   读取文件: {csv_files[0]} | 原始行数: {len(df_raw)}")

# 去除列名空格
df_raw.columns = df_raw.columns.str.strip()

# 1. 锁定 Target
target_col = None
for c in ['log(1o2)', 'Target', 'Activity', 'Y', 'pIC50']:
    if c in df_raw.columns: target_col = c; break
if not target_col:
    for c in df_raw.columns:
        if 'log' in c.lower() or 'target' in c.lower(): target_col = c; break

if not target_col: raise ValueError(f"❌ 无法识别 Target 列! 现有列: {list(df_raw.columns)}")

# 2. 锁定 SMILES
smiles_col = 'SMILES' if 'SMILES' in df_raw.columns else df_raw.columns[0]
print(f"   Target: '{target_col}' | SMILES: '{smiles_col}'")

# 3. 初步清洗: 去除 Target 缺失行
init_len = len(df_raw)
df_raw = df_raw.dropna(subset=[target_col])
# 强制转为数值，无法转换的变为 NaN 然后删掉
df_raw[target_col] = pd.to_numeric(df_raw[target_col], errors='coerce')
df_raw = df_raw.dropna(subset=[target_col])

if len(df_raw) < init_len:
    print(f"   ⚠️ 剔除了 {init_len - len(df_raw)} 行无效 Target 数据")

# ========================================================
# [Step 2] RDKit 处理 (Canonicalize + Aggregation)
# ========================================================
print("\n>>> [Step 2] RDKit Processing (Canonicalization & Aggregation)...")

desc_names = [x[0] for x in Descriptors._descList]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)

valid_rows = []
invalid_smiles = 0

for idx, row in df_raw.iterrows():
    smi = row[smiles_col]
    val = row[target_col]

    # 防御: 确保 smi 是字符串
    if not isinstance(smi, str):
        invalid_smiles += 1
        continue

    try:
        mol = Chem.MolFromSmiles(smi)
        if mol:
            # 标准化
            canon_smi = Chem.MolToSmiles(mol, isomericSmiles=True)
            # 计算特征
            feats = list(calc.CalcDescriptors(mol))

            valid_rows.append({
                'SMILES_Canonical': canon_smi,
                'Target_Raw': val,
                'Features': feats
            })
        else:
            invalid_smiles += 1
    except:
        invalid_smiles += 1
        continue

print(f"   - 原始有效分子数: {len(valid_rows)}")
print(f"   - 无效/解析失败 SMILES: {invalid_smiles}")

# ★★★ 核心改进: 结构聚合 (Aggregation) ★★★
# 如果同一个 SMILES 出现多次，取 Target 的平均值
df_temp = pd.DataFrame(valid_rows)
# 按 SMILES_Canonical 分组，Target 取均值，特征取第一条(理论上特征应该是一样的)
# 注意：特征列是 list，不能直接 groupby，我们先聚合 Target
df_agg = df_temp.groupby('SMILES_Canonical').agg({
    'Target_Raw': 'mean',
    'Features': 'first' # 特征对于同一结构是固定的
}).reset_index()

print(f"   - 聚合后唯一结构数: {len(df_agg)} (合并了 {len(valid_rows) - len(df_agg)} 个重复结构)")

# 展开特征列
# 将 Features (list) 拆分成多列
feat_data = pd.DataFrame(df_agg['Features'].tolist(), columns=desc_names)
# 合并回去
df_features = pd.concat([
    df_agg[['Target_Raw', 'SMILES_Canonical']],
    feat_data
], axis=1)

# 重命名以符合后续流程
df_features.rename(columns={
    'Target_Raw': 'Target_Log1o2',
    'SMILES_Canonical': 'SMILES_Meta'
}, inplace=True)

# 初步清洗无穷大
df_features.replace([np.inf, -np.inf], np.nan, inplace=True)

# ========================================================
# [Step 3] 动态分层切分 (Robust Stratified Split)
# ========================================================
print("\n>>> [Step 3] Stratified Splitting...")

# 读取种子
seed_file = "current_seed_config.txt"
if os.path.exists(seed_file):
    try: CURRENT_SEED = int(open(seed_file).read().strip())
    except: CURRENT_SEED = 288
else: CURRENT_SEED = 288
print(f"   Seed: {CURRENT_SEED}")

X = df_features.drop(columns=['Target_Log1o2', 'SMILES_Meta'])
y = df_features['Target_Log1o2']
meta = df_features['SMILES_Meta']

# ★ 改进: 健壮的分箱逻辑 ★
n_samples = len(y)
# 至少要有 5 个样本才能分 2 箱，否则分箱无意义
num_bins = min(10, int(n_samples / 5))

if num_bins < 2:
    print("   ⚠️ 样本过少，无法分层，回退到随机切分")
    stratify_col = None
else:
    try:
        # qcut 可能会因为数据重复导致 bin 边界重叠，使用 duplicates='drop'
        stratify_col = pd.qcut(y, q=num_bins, labels=False, duplicates='drop')
        # 再次检查：如果 drop 后只剩 1 个 bin，也无法分层
        if stratify_col.nunique() < 2:
            print("   ⚠️ Target 分布过于集中，无法有效分层，回退到随机切分")
            stratify_col = None
    except:
        stratify_col = None

# 组合以便切分
data_full = pd.concat([y, meta, X], axis=1)

# 第一刀 (Test 15%)
try:
    train_val, test = train_test_split(data_full, test_size=0.15, random_state=CURRENT_SEED, stratify=stratify_col)
except ValueError:
    # 如果某个类别的样本太少(比如只有1个)，stratify 会报错
    print("   ⚠️ 分层切分失败 (某类别样本不足)，回退到随机切分")
    train_val, test = train_test_split(data_full, test_size=0.15, random_state=CURRENT_SEED)

# 重新为 Train+Val 生成分层依据
if stratify_col is not None:
    y_tv = train_val['Target_Log1o2']
    try:
        num_bins_tv = min(10, int(len(y_tv) / 5))
        if num_bins_tv >= 2:
            stratify_tv = pd.qcut(y_tv, q=num_bins_tv, labels=False, duplicates='drop')
        else: stratify_tv = None
    except: stratify_tv = None
else:
    stratify_tv = None

# 第二刀 (Val 15% of Total -> ~17.6% of TrainVal)
try:
    train, val = train_test_split(train_val, test_size=15/85, random_state=CURRENT_SEED, stratify=stratify_tv)
except:
    train, val = train_test_split(train_val, test_size=15/85, random_state=CURRENT_SEED)

# 拆解函数
def split_xy_meta(df):
    return df.drop(columns=['Target_Log1o2', 'SMILES_Meta']), df['Target_Log1o2'], df['SMILES_Meta']

X_train, y_train, s_train = split_xy_meta(train)
X_val, y_val, s_val = split_xy_meta(val)
X_test, y_test, s_test = split_xy_meta(test)

print(f"   Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# ========================================================
# [Step 4] 严谨清洗 (Fit on Train, Keep DataFrame)
# ========================================================
print("\n>>> [Step 4] Rigorous Cleaning (Fit on Train)...")

# 1. Imputer (保护 Index)
# ----------------------------------------------------
print("   -> Imputing...")
imputer = SimpleImputer(strategy='median')
# Fit
imputer.fit(X_train)

# Transform 并立刻还原为 DataFrame (保留列名和索引)
def impute_df(X_df, imp):
    data_imputed = imp.transform(X_df)
    return pd.DataFrame(data_imputed, columns=X_df.columns, index=X_df.index)

X_train = impute_df(X_train, imputer)
X_val   = impute_df(X_val, imputer)
X_test  = impute_df(X_test, imputer)

# 2. 常量过滤 (直接判断 nunique)
# ----------------------------------------------------
# 比 VarianceThreshold 更直观，浮点数不敏感
print("   -> Removing Constants...")
# 找出 Train 中只有一个值的列
const_cols = [c for c in X_train.columns if X_train[c].nunique() <= 1]

if const_cols:
    X_train.drop(columns=const_cols, inplace=True)
    X_val.drop(columns=const_cols, inplace=True)
    X_test.drop(columns=const_cols, inplace=True)
    print(f"      Deleted {len(const_cols)} constant features")

# 3. 高相关过滤 (Pandas Corr)
# ----------------------------------------------------
print("   -> Removing High Correlation (>0.90)...")
# 只在 Train 上计算
corr_matrix = X_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

if to_drop:
    X_train.drop(columns=to_drop, inplace=True)
    X_val.drop(columns=to_drop, inplace=True)
    X_test.drop(columns=to_drop, inplace=True)
    print(f"      Deleted {len(to_drop)} correlated features")

print(f"   ✅ 最终特征数: {X_train.shape[1]}")

# ========================================================
# [Step 5] 保存
# ========================================================
os.makedirs('data/processed', exist_ok=True)
os.makedirs('results/logs', exist_ok=True)

# 保存数据
def save_data(X, y, s, name):
    out = X.copy()
    out['Target_Log1o2'] = y
    out['SMILES_Meta'] = s
    out.to_csv(f'data/processed/{name}', index=False)

save_data(X_train, y_train, s_train, 'train.csv')
save_data(X_val, y_val, s_val, 'val.csv')
save_data(X_test, y_test, s_test, 'test.csv')

# 保存一份元数据日志 (记录删除了哪些列)
log_data = {
    "final_features": list(X_train.columns),
    "dropped_constant": const_cols,
    "dropped_correlation": to_drop
}
with open('results/logs/feature_log.json', 'w') as f:
    json.dump(log_data, f, indent=4)

print(f"\n💾 数据已保存至 data/processed/")
print(f"📝 特征日志已保存至 results/logs/feature_log.json")

🚀 启动 RDKit 特征工程 (Robust Mode) | 目录: D:\HO\my_smiles_project\y_g.baby\project_rdkt

>>> [Step 1] Loading Raw Data...
   读取文件: cdft_qsar.CSV | 原始行数: 151
   Target: 'log(1o2)' | SMILES: 'SMILES'

>>> [Step 2] RDKit Processing (Canonicalization & Aggregation)...


   - 原始有效分子数: 151
   - 无效/解析失败 SMILES: 0
   - 聚合后唯一结构数: 151 (合并了 0 个重复结构)

>>> [Step 3] Stratified Splitting...
   Seed: 288
   Train: (105, 217), Val: (23, 217), Test: (23, 217)

>>> [Step 4] Rigorous Cleaning (Fit on Train)...
   -> Imputing...
   -> Removing Constants...
      Deleted 40 constant features
   -> Removing High Correlation (>0.90)...
      Deleted 51 correlated features
   ✅ 最终特征数: 126

💾 数据已保存至 data/processed/
📝 特征日志已保存至 results/logs/feature_log.json
